# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [42]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from g_config import gcode
from w_config import wcode

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [43]:
path = '..\WeatherPy\Weather_cities.csv'
df_cities = pd.read_csv(path)
display(df_cities.head())



,City,Latitude,Longitude,Max temp,Humidity,Cloudiness,Wind speed,Country,Date
0,barrow,71.2906,-156.7887,-22.00,77,90,6.17,US,1613256569
1,ushuaia,-54.8000,-68.3000,7.00,81,75,0.89,AR,1613256576
2,nikolskoye,59.7035,30.7861,-8.00,93,75,2.00,RU,1613256610
3,mataura,-46.1927,168.8643,19.44,64,100,3.13,NZ,1613256572
4,omsukchan,62.5333,155.8000,-31.57,83,84,1.95,RU,1613256625


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [44]:
# Configuring Gmaps
gmaps.configure(api_key=gcode)
figure_layout = {
    'width': '1000px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [45]:
# Settin Coordinates
cities = df_cities[['Latitude', 'Longitude']]
cities_test = cities.head() # Test variable to check against map
display(cities_test)
print(cities_test.dtypes)

# Setting humidity for weights
humidity = df_cities['Humidity'].astype(float)
humidity_test = humidity.head() # Test variable to check against map
display(humidity_test)
print(humidity_test.dtypes)

,Latitude,Longitude
0,71.2906,-156.7887
1,-54.8000,-68.3000
2,59.7035,30.7861
3,-46.1927,168.8643
4,62.5333,155.8000


Latitude     float64
Longitude    float64
dtype: object


0    77.0
1    81.0
2    93.0
3    64.0
4    83.0
Name: Humidity, dtype: float64

float64


In [46]:
# Creating the heat layer

heat_layer = gmaps.heatmap_layer(cities, weights = humidity,
                                dissipating = False, max_intensity = 100,
                                point_radius = 2)
fig.add_layer(heat_layer)

In [47]:
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Metric system is being used here.
Ideal conditions for next vacations:

- Humidity between 45% - 60%
- Temperature between 25° and 32° celsius
- Cloudiness between 0 and 40 kph

In [50]:
display(df_cities.head())

,City,Latitude,Longitude,Max temp,Humidity,Cloudiness,Wind speed,Country,Date
0,barrow,71.2906,-156.7887,-22.00,77,90,6.17,US,1613256569
1,ushuaia,-54.8000,-68.3000,7.00,81,75,0.89,AR,1613256576
2,nikolskoye,59.7035,30.7861,-8.00,93,75,2.00,RU,1613256610
3,mataura,-46.1927,168.8643,19.44,64,100,3.13,NZ,1613256572
4,omsukchan,62.5333,155.8000,-31.57,83,84,1.95,RU,1613256625


In [53]:
ideal_vac = df_cities[(df_cities['Humidity']>=45) & (df_cities['Humidity']<60) & 
          (df_cities['Max temp']>=25) & (df_cities['Max temp']<32) & 
          (df_cities['Cloudiness']>=0) & (df_cities['Cloudiness']<40)]
display(ideal_vac)
len(ideal_vac['City'].unique())

,City,Latitude,Longitude,Max temp,Humidity,Cloudiness,Wind speed,Country,Date
92,santa cruz,-17.8000,-63.1667,30.00,58,20,3.09,BO,1613256590
104,mandera,3.9366,41.8670,25.46,45,5,6.36,KE,1613256647
120,pisco,-13.7000,-76.2167,26.00,57,0,10.80,PE,1613256624
134,cabo san lucas,22.8909,-109.9124,26.00,56,1,4.12,MX,1613257147
383,petatlan,17.5167,-101.2667,29.00,54,1,2.57,MX,1613257323
397,buin,-33.7333,-70.7500,25.00,50,0,3.09,CL,1613257328
425,pochutla,15.7432,-96.4661,30.00,54,20,10.29,MX,1613257338
533,portobelo,9.5500,-79.6500,31.00,48,20,4.63,PA,1613257376


8

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [131]:
for hotel in ideal_vac.iterrows():
    print(hotel[1][1])
    lat = hotel[1][1]
    print(hotel[1][2])
    lon = hotel[1][2]
    coordinates = f'{str(lat)}, {str(lon)}'
    print(coordinates)
    print(type(coordinates))
    print('------')
    break

-17.8
-63.1667
-17.8, -63.1667
<class 'str'>
------


In [134]:
from pprint import pprint

#coordinates_test = '22.8909, -109.9124' This was used to test the outcome for the google places response

hotel_data = [] # This is the list where name and coordinates from the Hotels will be appended

# Fixed parameters
target_search = 'Hotels'
target_radius = 5000

# Url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Iteration over ideal_vac dataframe

for hotel in ideal_vac.iterrows():
    
    lat = hotel[1][1]
    lon = hotel[1][2]
    hotel_city = hotel[1][0]
    hotel_country = hotel[1][7]
    coordinates = f'{str(lat)}, {str(lon)}' 
    params = {
    'location': coordinates,
    'keyword': target_search,
    'radius': target_radius,
    'key': gcode}
    
    try:
        response = requests.get(base_url, params=params).json()
        hotel_lat = response['results'][1]['geometry']['location']['lat']
        hotel_lon = response['results'][1]['geometry']['location']['lng']
        hotel_name = response['results'][1]['name']


        hotel_data.append([hotel_name, hotel_city, hotel_country, hotel_lat, hotel_lon])
        print(f'Hotel name: {hotel_name}\n City:{hotel_city}\n Country:{hotel_country}\n Latitude: {hotel_lat}\n Longitude: {hotel_lon}')
        print('----------------------')
    except IndexError:
        
        print(f'There was a problem trying to find with index 1. Index 0 used instead. The result is:')      
        try:
            response = requests.get(base_url, params=params).json()
            hotel_lat = response['results'][0]['geometry']['location']['lat']
            hotel_lon = response['results'][0]['geometry']['location']['lng']
            hotel_name = response['results'][0]['name']
            hotel_data.append([hotel_name, hotel_city, hotel_country, hotel_lat, hotel_lon])
            
            print(f'Hotel name: {hotel_name}\n City:{hotel_city}\n Country:{hotel_country}\n Latitude: {hotel_lat}\n Longitude: {hotel_lon}')
            print('----------------------')
        except IndexError:
            print('No hotel was found near to this city')
            print('----------------------')

print(hotel_data)
    

Hotel name: TUCABACA HOTEL | Boutique
 City:santa cruz
 Country:BO
 Latitude: -17.7747764
 Longitude: -63.17031169999999
----------------------
Hotel name: Hanan Guest House
 City:mandera
 Country:KE
 Latitude: 3.9352985
 Longitude: 41.8556529
----------------------
Hotel name: Hostal Mandala
 City:pisco
 Country:PE
 Latitude: -13.7098697
 Longitude: -76.20645979999999
----------------------
Hotel name: Hotel Riu Palace Cabo San Lucas
 City:cabo san lucas
 Country:MX
 Latitude: 22.897335
 Longitude: -109.8917105
----------------------
Hotel name: Hotel California Petatlán
 City:petatlan
 Country:MX
 Latitude: 17.5376542
 Longitude: -101.2688742
----------------------
There was a problem trying to find with index 1. Index 0 used instead. The result is:
Hotel name: Paihuen
 City:buin
 Country:CL
 Latitude: -33.7188826
 Longitude: -70.7335867
----------------------
There was a problem trying to find with index 1. Index 0 used instead. The result is:
No hotel was found near to this city
--

### This section is just to overview the specific data from the response

In [122]:
# hotel_lat = response['results'][1]['geometry']['location']['lat']
# hotel_lon = response['results'][1]['geometry']['location']['lng']
# pprint(response)
# pprint(response)

In [136]:
# Converting Hotel data into a Dataframe

hotel_df = pd.DataFrame(hotel_data, columns = ['Hotel', 'City', 'Country', 'Latitude', 'Longitude'])
hotel_df

,Hotel,City,Country,Latitude,Longitude
0,TUCABACA HOTEL | Boutique,santa cruz,BO,-17.774776,-63.170312
1,Hanan Guest House,mandera,KE,3.935299,41.855653
2,Hostal Mandala,pisco,PE,-13.709870,-76.206460
3,Hotel Riu Palace Cabo San Lucas,cabo san lucas,MX,22.897335,-109.891711
4,Hotel California Petatlán,petatlan,MX,17.537654,-101.268874
5,Paihuen,buin,CL,-33.718883,-70.733587
6,Buenaventura,portobelo,PA,9.532285,-79.668649


In [138]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [139]:
# Add marker layer ontop of heat map
hotel_list = hotel_df["Hotel"].tolist()
marker_locations = hotel_df[['Latitude', 'Longitude']]

markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Hotels {name}" for name in hotel_list])
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…